In [1]:
import os
import cv2
import numpy as np
import glob
import json
import pandas as pd
import zipfile

In [2]:
DATA_PATH = os.path.join('..','data')
LABEL_PATH = os.path.join(DATA_PATH, 'label_data')
FRAMES_PATH = os.path.join(DATA_PATH, 'frames')
VIDEOS_PATH = os.path.join(DATA_PATH, 'videos')

In [12]:
videos_list = os.listdir(VIDEOS_PATH)
videos_list

['120210923000455_551.avi',
 '120210923001407_550.avi',
 '120210923002318_550.avi',
 '120210923003229_552.avi',
 '120210923083827_568.avi',
 '120210923084756_567.avi',
 '120210923085724_564.avi',
 '120210923090649_565.avi',
 '120210923091615_562.avi',
 '120210923102215_567.avi',
 '120210923103143_564.avi',
 '120210923104108_565.avi',
 '120210923105034_563.avi',
 '120210923111858_569.avi',
 '120210923112828_567.avi',
 '120210923113755_567.avi',
 '120210923114723_570.avi',
 '120210923115654_575.avi',
 '120210923120630_567.avi',
 '120210923121558_575.avi',
 '120210923122534_578.avi',
 '120210923123513_571.avi',
 'frames']

In [60]:
# path = path = os.path.join(DATA_PATH, videos_list[0])
# frames_path = os.path.join(DATA_PATH, 'frames', videos_list[0])

Разбивка видео на кадры

'..\\data\\ПРОЕКТ Х\\120210923123513_571.avi'

In [31]:
frames_type = '.jpg'

frames_dir = os.path.join(VIDEOS_PATH, 'frames')
videos_dir = VIDEOS_PATH

for video_name in videos_list:
    path = os.path.join(videos_dir, video_name)
#     Путь без типа видео
    frames_path = os.path.join(frames_dir, video_name)[:-4]
#     Поворот
#     os.system(f'ffmpeg -i {path} -vf "transpose=2,transpose=2" {path[:-4]}_rotated{path[-4:]}')
    # Special command
    # See: https://stackoverflow.com/questions/10225403/how-can-i-extract-a-good-quality-jpeg-image-from-a-video-file-with-ffmpeg
#     os.system(f'ffmpeg -i {path[:-4]}_rotated{path[-4:]} -qscale:v 2 {frames_path[:-4]}_%d.jpg')
    os.system(f'ffmpeg -i {path} -qscale:v 2 {frames_path}_%d{frames_type}')

In [28]:
# src = glob.glob(os.path.join(frames_dir, '*.jpg'))[1_000]
# img = cv2.imread(src)
# img = cv2.rotate(img, cv2.ROTATE_180)
# cv2.imwrite('D:\example.jpg', img)

True

In [ ]:
Поворот картинок на 180 градусов

In [30]:
# import cv2

# for frame_path in glob.glob(os.path.join(frames_dir, '*.jpg')):
#     src = cv2.imread(frame_path)
#     frame = cv2.rotate(src, cv2.ROTATE_180)
#     cv2.imwrite(frame_path, frame)

KeyboardInterrupt: 

Создание csv файла меток

In [3]:
json_list = glob.glob(os.path.join(FRAMES_PATH, '*.json'))

In [4]:
path, ladle_label = [], []
for fn in json_list:
    with open(fn, 'r') as f:
        data = json.load(f)
        if data['flags']['ladle']==True or data['flags']['no_ladle']==True:
            path.append(data['imagePath'])
            ladle_label.append(1 if data['flags']['ladle']==True else 0)

In [5]:
data_df = pd.DataFrame(data = {'path': path, 'ladle_label': ladle_label})
data_df.head(5)

,path,ladle_label
0,120210923001407_550_1.jpg,0
1,120210923001407_550_10.jpg,0
2,120210923001407_550_100.jpg,1
3,120210923001407_550_1000.jpg,1
4,120210923001407_550_1001.jpg,1


In [6]:
data_df.to_csv(os.path.join(DATA_PATH,'label_data', 'data_df.csv'))

In [7]:
data_df

,path,ladle_label
0,120210923001407_550_1.jpg,0
1,120210923001407_550_10.jpg,0
2,120210923001407_550_100.jpg,1
3,120210923001407_550_1000.jpg,1
4,120210923001407_550_1001.jpg,1
...,...,...
4526,120210923002318_550_710.jpg,1
4527,120210923002318_550_711.jpg,1
4528,120210923002318_550_712.jpg,1
4529,120210923002318_550_713.jpg,1


Архивация

In [16]:
with zipfile.ZipFile(os.path.join(LABEL_PATH, 'data.zip'), 'w') as zip_ref:
    for index, row in data_df.iterrows():
        zip_ref.write(os.path.join(FRAMES_PATH, row['path']), arcname = os.path.join('data', 'frames', row['path']))
    
    zip_ref.write(os.path.join(LABEL_PATH, 'data_df.csv'), arcname = os.path.join('data', 'labels', 'data_df.csv'))